# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
A. Gould  ->  A. Gould  |  ['A. Gould']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']


Arxiv has 50 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.03936


extracting tarball to tmp_2309.03936...

 done.


S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


...

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
A. Gould  ->  A. Gould  |  ['A. Gould']


Found 79 bibliographic references in tmp_2309.03944/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2309.04403


extracting tarball to tmp_2309.04403...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2309.04403/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.03936-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.03936) | **No strong radio absorption detected in the low-frequency spectra of  radio-loud quasars at z > 5.6**  |
|| A. J. Gloudemans, et al. -- incl., <mark>S. Belladitta</mark> |
|*Appeared on*| *2023-09-11*|
|*Comments*| *15 pages, 7 figures, accepted for publication in A&A*|
|**Abstract**| We present the low-frequency radio spectra of 9 high-redshift quasars at $5.6 \leq z \leq 6.6$ using the Giant Metre Radio Telescope band-3, -4, and -5 observations ($\sim$300-1200 MHz), archival Low Frequency Array (LOFAR; 144 MHz), and Very Large Array (VLA; 1.4 and 3 GHz) data. Five of the quasars in our sample have been discovered recently, representing some of the highest redshift radio bright quasars known at low-frequencies. We model their radio spectra to study their radio emission mechanism and age of the radio jets by constraining the spectral turnover caused by synchrotron self-absorption (SSA) or free-free absorption (FFA). Besides J0309+2717, a blazar at $z=6.1$, our quasars show no sign of a spectral flattening between 144 MHz and a few GHz, indicating there is no strong SSA or FFA absorption in the observed frequency range. However, we find a wide range of spectral indices between $-1.6$ and $0.05$, including the discovery of 3 potential ultra-steep spectrum quasars. Using further archival VLBA data, we confirm that the radio SED of the blazar J0309+2717 likely turns over at a rest-frame frequency of 0.6-2.3 GHz (90-330 MHz observed frame), with a high-frequency break indicative of radiative ageing of the electron population in the radio lobes. Ultra-low frequency data below 50 MHz are necessary to constrain the absorption mechanism for J0309+2717 and the turnover frequencies for the other high-$z$ quasars in our sample. A relation between linear radio jet size and turnover frequency has been established at low redshifts. If this relation were to hold at high redshifts, the limits on the turnover frequency of our sample suggest the radio jet sizes must be more extended than the typical sizes observed in other radio-bright quasars at similar redshift. To confirm this deep radio follow-up observations with high spatial resolution are required. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.03944-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.03944) | **Gaia22dkvLb: A Microlensing Planet Potentially Accessible to  Radial-Velocity Characterization**  |
|| Z. Wu, et al. -- incl., <mark>K. El-Badry</mark>, <mark>A. Gould</mark> |
|*Appeared on*| *2023-09-11*|
|*Comments*| **|
|**Abstract**| We report discovering an exoplanet from following up a microlensing event alerted by Gaia. The event Gaia22dkv is toward a nearby disk source at ~2.5 kpc rather than the traditional bulge microlensing fields. Our primary analysis yields a Jovian planet with M_p = 0.50 +/- 0.05 M_J at a projected orbital separation r_perp = 1.63 +/- 0.17 AU. The host is a turnoff star with mass 1.24 +/- 0.06 M_sun and distance of 1.35 +/- 0.09 kpc, and at r'~14, it is far brighter than any previously discovered microlensing planet host, opening up the opportunity of testing the microlensing model with radial velocity (RV) observations. RV data can be used to measure the planet's orbital period and eccentricity, and they also enable searching for inner planets of the microlensing cold Jupiter, as expected from the "inner-outer correlation" inferred from Kepler and RV discoveries. Furthermore, we show that Gaia astrometric microlensing will not only allow precise measurements of its angular Einstein radius theta_E, but also directly measure the microlens parallax vector and unambiguously break a geometric light-curve degeneracy, leading to definitive characterization of the lens system. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.04403-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.04403) | **VLTI/GRAVITY Observations and Characterization of the Brown Dwarf  Companion HD 72946 B**  |
|| W. O. Balmer, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2023-09-11*|
|*Comments*| *Accepted for publication in the Astrophysical Journal. 32 pages, 14 figures, 7 tables*|
|**Abstract**| Tension remains between the observed and modeled properties of substellar objects, but objects in binary orbits, with known dynamical masses can provide a way forward. HD 72946 B is a recently imaged brown dwarf companion to the nearby, solar type star. We achieve $\sim100~\mu\mathrm{as}$ relative astrometry of HD 72946 B in the K-band using VLTI/GRAVITY, unprecedented for a benchmark brown dwarf. We fit an ensemble of measurements of the orbit using orbitize! and derive a strong dynamical mass constraint $\mathrm{M_B}=69.5\pm0.5~\mathrm{M_{Jup}}$ assuming a strong prior on the host star mass $\mathrm{M_A}=0.97\pm0.01~\mathrm{M_\odot}$ from an updated stellar analysis. We fit the spectrum of the companion to a grid of self-consistent BT-Settl-CIFIST model atmospheres, and perform atmospheric retrievals using petitRADTRANS. A dynamical mass prior only marginally influences the sampled distribution on effective temperature, but has a large influence on the surface gravity and radius, as expected. The dynamical mass alone does not strongly influence retrieved pressure-temperature or cloud parameters within our current retrieval setup. Independent of cloud prescription and prior assumptions, we find agreement within $\pm2\,\sigma$ between the C/O ratio of the host ($0.52\pm0.05)$ and brown dwarf ($0.43$ to $0.63$), as expected from a molecular cloud collapse formation scenario, but our retrieved metallicities are implausibly high ($0.6-0.8$) in light of an excellent agreement of the data with the solar abundance model grid. Future work on our retrieval framework will seek to resolve this tension. Additional study of low surface-gravity objects is necessary to assess the influence of a dynamical mass prior on atmospheric analysis. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.03936.md
    + _build/html/tmp_2309.03936/./linear_size_vs_turnover_v10.png
    + _build/html/tmp_2309.03936/./J0309_radio_spec_v8.png
    + _build/html/tmp_2309.03936/./J172_curved_spec_v6.png
    + _build/html/tmp_2309.03936/./radio_luminosities_from_lit_v3.png
exported in  _build/html/2309.03944.md
    + _build/html/tmp_2309.03944/figures/host_info.png
    + _build/html/tmp_2309.03944/figures/RV.png
    + _build/html/tmp_2309.03944/figures/Galactic.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\RNum}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$
$\newcommand{\arraystretch}{2.0}$</div>



<div id="title">

# No strong radio absorption detected in the low-frequency spectra of radio-loud quasars at $z>5.6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.03936-b31b1b.svg)](https://arxiv.org/abs/2309.03936)<mark>Appeared on: 2023-09-11</mark> -  _15 pages, 7 figures, accepted for publication in A&A_

</div>
<div id="authors">

A. J. Gloudemans, et al. -- incl., <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** We present the low-frequency radio spectra of 9 high-redshift quasars at $5.6 \leq z \leq 6.6$ using the Giant Metre Radio Telescope band-3, -4, and -5 observations ( $\sim$ 300-1200 MHz), archival Low Frequency Array (LOFAR; 144 MHz), and Very Large Array (VLA; 1.4 and 3 GHz) data. Five of the quasars in our sample have been discovered recently, representing some of the highest redshift radio bright quasars known at low-frequencies. We model their radio spectra to study their radio emission mechanism and age of the radio jets by constraining the spectral turnover caused by synchrotron self-absorption (SSA) or free-free absorption (FFA). Besides J0309+2717, a blazar at $z=6.1$ , our quasars show no sign of a spectral flattening between 144 MHz and a few GHz, indicating there is no strong SSA or FFA absorption in the observed frequency range. However, we find a wide range of spectral indices between $-1.6$ and $0.05$ , including the discovery of 3 potential ultra-steep spectrum quasars. Using further archival VLBA data, we confirm that the radio SED of the blazar J0309+2717 likely turns over at a rest-frame frequency of 0.6-2.3 GHz (90-330 MHz observed frame), with a high-frequency break indicative of radiative ageing of the electron population in the radio lobes. Ultra-low frequency data below 50 MHz are necessary to constrain the absorption mechanism for J0309+2717 and the turnover frequencies for the other high- $z$ quasars in our sample.A relation between linear radio jet size and turnover frequency has been established at low redshifts. If this relation were to hold at high redshifts, the limits on the turnover frequency of our sample suggest the radio jet sizes must be more extended than the typical sizes observed in other radio-bright quasars at similar redshift. To confirm this deep radio follow-up observations with high spatial resolution are required.

</div>

<div id="div_fig1">

<img src="tmp_2309.03936/./linear_size_vs_turnover_v10.png" alt="Fig3" width="100%"/>

**Figure 3. -** Linear sizes versus rest-frame turnover frequency for the quasars in our sample, J172+18, and the 10 high-$z$ quasars from [Shao, Wagg and Wang (2022)](). The grey points show low-redshift ($z\approx0.1$-2) peaked spectrum sources from [Keim, Callingham and Röttgering (2019)](), which follow a tight relation given by the black solid line (see  ([Keim, Callingham and Röttgering 2019]()) ). Apart from J0309+2717, we have only upper limits on the linear sizes of the quasars in our sample and their turnover frequencies. Five of the quasars from [Shao, Wagg and Wang (2022)]() have been observed at mas resolution with the VLBA and three of them have size measurements, which seem to follow the low-$z$ relation. The solid and dashed grey lines show the frequencies of the LoLSS and LoDeSS survey, respectively, and the black open circles the resolution of the LOFAR VLBI and VLBA at 150 MHz and 1.6 GHz (observed frame). (*fig:linear_sizes*)

</div>
<div id="div_fig2">

<img src="tmp_2309.03936/./J0309_radio_spec_v8.png" alt="Fig5.1" width="50%"/><img src="tmp_2309.03936/./J172_curved_spec_v6.png" alt="Fig5.2" width="50%"/>

**Figure 5. -**  Radio spectra of J0309+2717 (left) and J172+18 (right) including reported literature radio flux density measurements. Left: The GMRT subband flux densities of the band 3 and 4 observations are shown in light purple. The VLBA and VLA measurements have been provided by [Spingola, Dallacasa and Belladitta (2020)](). The dashed, dash-dotted and dotted line show the best fitting homogeneous SSA, homogeneous FFA, and internal FFA models, respectively. The homogeneous FFA with a break due to an ageing electron population (grey line) gives the best fit to the spectrum with a reduced $\chi^2$ of 0.6 and $\Delta$BIC $>6$ compared to all other models. Randomly selected MCMC fits are shown in light grey to visualize the uncertainty in the model fit with the parameters given in Tab. \ref{tab:curve_parameters}. The bottom plot shows the residual of the homogeneous FFA model with a break. Right: The VLA-L and -S observations have been conducted by [Bañados, Mazzucchelli and Momjian (2021)]() and the VLBA observations by [Momjian, et. al (2021)](). Upper limits are given for the LOFAR at 144 MHz and the VLBA observations at 4.67 and 7.67 GHz. Due to a limited number of data points between 150 MHz and 1 GHz the absorption models remain unconstrained. However, we can conclude that the spectrum likely peaks between 0.25-1.3 GHz (rest-frame 2-10 GHz), meaning it can be classified as a GPS source. Again the residuals are shown in the bottom panel.  (*fig:radio_curves*)

</div>
<div id="div_fig3">

<img src="tmp_2309.03936/./radio_luminosities_from_lit_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** Radio luminosities at 144 MHz versus spectroscopic redshift of our selected sample and other known LOFAR detected quasars compiled by [Gloudemans, Duncan and Röttgering (2021)](), [Gloudemans, Duncan and Saxena (2022)](). Our sample probes some of the brightest and highest redshift radio-loud quasars currently known at low frequencies. The heavily obscured AGN COS-87259 is currently the highest redshift radio-loud quasar known and has been detected by LOFAR  ([Endsley, Stark and Bouwens 2022](), [Endsley, Stark and Lyu 2022]()) . The two radio-loud quasars around $z\sim6.1$(J1427+3312 and J1429+5447) have not been included in our sample because their radio spectra have already been studied by [Shao, Wagg and Wang (2020)]() and [Shao, Wagg and Wang (2022)](), with J1427+3312 showing a turnover around 1.7 GHz rest-frame.  (*fig:radio_lum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.03936"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\bdv}[1]{\mbox{\boldmath#1}}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\sinc}{{\rm sinc}}$
$\newcommand{\kms}{{\rm km} {\rm s}^{-1}}$
$\newcommand{\masyr}{{\rm mas} {\rm yr}^{-1}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\sat}{{\rm sat}}$
$\newcommand{\muas}{\mu{\rm as}}$
$\newcommand{\var}{{\rm var}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\orb}{{\rm orb}}$
$\newcommand{\obs}{{\rm obs}}$
$\newcommand{\max}{{\rm max}}$
$\newcommand{\min}{{\rm min}}$
$\newcommand{\rel}{{\rm rel}}$
$\newcommand{\ast}{{\rm ast}}$
$\newcommand{\eff}{{\rm eff}}$
$\newcommand{\rot}{{\rm rot}}$
$\newcommand{\lsr}{{\rm lsr}}$
$\newcommand{\hel}{{\rm hel}}$
$\newcommand{\geo}{{\rm geo}}$
$\newcommand{\B}{{\rm B}}$
$\newcommand{\Sc}{{\rm S}}$
$\newcommand{\L}{{\rm L}}$
$\newcommand{\E}{{\rm E}}$
$\newcommand{\bpi}{{\bdv\pi}}$
$\newcommand{\bmu}{{\bdv\mu}}$
$\newcommand{\balpha}{{\bdv\alpha}}$
$\newcommand{\bgamma}{{\bdv\gamma}}$
$\newcommand{\bDelta}{{\bdv\Delta}}$
$\newcommand{\btheta}{{\bdv\theta}}$
$\newcommand{\bphi}{{\bdv\phi}}$
$\newcommand{\bp}{{\bf p}}$
$\newcommand{\bv}{{\bf v}}$
$\newcommand{\bu}{{\bf u}}$
$\newcommand{\naive}{{\rm naive}}$
$\newcommand{\revise}{\bf}$
$\newcommand{\revisec}{\color{red}}$
$\newcommand{\rowmac}{#1}$</div>



<div id="title">

# Gaia22dkvLb: A Microlensing Planet Potentially Accessible to Radial-Velocity Characterization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.03944-b31b1b.svg)](https://arxiv.org/abs/2309.03944)<mark>Appeared on: 2023-09-11</mark> - 

</div>
<div id="authors">

Z. Wu, et al. -- incl., <mark>K. El-Badry</mark>, <mark>A. Gould</mark>

</div>
<div id="abstract">

**Abstract:** We report discovering an exoplanet from following up a microlensing event alerted by Gaia. The event Gaia22dkv is toward a nearby disk source at $\sim 2.5$ kpc rather than the traditional bulge microlensing fields. Our primary analysis yields a Jovian planet with $M_{\rm p} = 0.50 \pm 0.05 M_{\rm J}$ at a projected orbital separation $r_{\perp} = 1.63\pm0.17$ AU. The host is a turnoff star with mass $1.24\pm{0.06} M_\odot$ and distance of $1.35\pm0.09$ kpc, and at $r'\approx14$ , it is far brighter than any previously discovered microlensing planet host, opening up the opportunity of testing the microlensing model with radial velocity (RV) observations.RV data can be used to measure the planet's orbital period and eccentricity, and they also enable searching for inner planets of the microlensing cold Jupiter, as expected from the "inner-outer correlation" inferred from $_ Kepler_$ and RV discoveries.Furthermore, we show that Gaia astrometric microlensing will not only allow precise measurements of its angular Einstein radius $\theta_{\rm E}$ , but also directly measure the microlens parallax vector and unambiguously break a geometric light-curve degeneracy, leading to definitive characterization of the lens system.

</div>

<div id="div_fig1">

<img src="tmp_2309.03944/figures/host_info.png" alt="Fig6" width="100%"/>

**Figure 6. -** Host properties of microlensing planets. We use the parameters of the default models (modeldef=1) in the microlensing planet catalog of the NASA Exoplanet Archive for all discoveries in the bulge field (black dots). The host properties of Kojima-1Lb (blue star) are from [Zang, Dong and Gould (2020)](), and those of Gaia22dkvLb are based on the globally best fit UPS$-$3.4$(u_0+)$ in this work. Panel A): Host  mass ($M_$\L$$) vs. distance ($D_$\L$$). Panel B):  Host brightness in $V$ band ($m_V$) vs. distance ($D_$\L$$). For the systems in the bulge field, we make crude $V$-band brightness estimates by assuming the lens star is a main-sequence star with stellar mass $M_$\L$$ at $D_$\L$$. We use the isochrone at a stellar age of $5$ Gyr from the Dartmouth Stellar Evolution Database   ([Dotter and Chaboyer 2008]())  to estimate the stellar luminosities. Then we apply the extinction corrections using the near-infrared extinction maps by [Gonzalez, Rejkuba and Zoccali (2012)]() and convert it to $A_V$ with the relationship in [Cardelli, Clayton and Mathis (1989)](). Panel C): The Galactic longitude ($l$) vs. latitude ($b$) distribution.
 (*fig:hosts*)

</div>
<div id="div_fig2">

<img src="tmp_2309.03944/figures/RV.png" alt="Fig3" width="100%"/>

**Figure 3. -** The upper panel displays the predicted radial velocity curves based on the orbital parameters of the best fit. The black lines represent the MCMC sample with $\Delta \chi^2 < 1$. The lower panel displays the RV semi amplitude $K$ and circular orbital period $P_{\rm circ}$ for all solutions. The best-fit solution is shown as a red filled circle with error bar, which is the only physically reasonable solution under the assumption that the blend is the lens and circular planetary orbital motion. Other solutions are shown as the color-coded open circles in red, yellow, green, blue, grey represents other solutions with $\Delta\chi^2 < 1, 4, 9, 16, 25$ compared with best fit, respectively. One year period is indicated as dashed line.
 (*fig:RV_curve*)

</div>
<div id="div_fig3">

<img src="tmp_2309.03944/figures/Galactic.png" alt="Fig4" width="100%"/>

**Figure 4. -** Galactic kinematics of the lens (green dot with error bar) and the source (blue dot with error bar) based on the best-fit solution.
Upper: the Galactocentric distance $R$ and height $z$ with respect to the Galactic plane positions (the red dot represents the Sun at $z_{\odot}$$=$ 25 pc;  ([ and Brooks 2008]()) ).
Lower:  the Toomre diagram. Dashed curves indicate constant peculiar velocities = 50, 100, 150, and 200 $\mathrm{km s^{-1}}$.
 (*fig:gala*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.03944"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

266  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
